# NSW boreholes with linked reports

This notebook demonstrates how to list NVCL boreholes from NSW and then use NSW's more detailed drilling_hyperspectral layer to link NVCL holes to reports.


In [6]:
from nvcl_kit.reader import NVCLReader
from nvcl_kit.param_builder import param_builder
import geopandas as gpd
import folium

param = param_builder('nsw', max_boreholes=5000)
if not param:
    print("Cannot build parameters for 'nsw'")

In [7]:
reader = NVCLReader(param)
if not reader.wfs:
    print("ERROR!")

In [8]:

# get more detail from NSW's drilling:drilling_drillholes_hyperspectral layer
wfs_url = "https://minview-qa.geoscience.nsw.gov.au/geoserver/drilling/wfs?service=WFS&version=1.1.0&request=GetFeature&typeName=drilling:drilling_drillholes_hyperspectral&outputFormat=json"

# Fetch data from the WFS server
gdf = gpd.read_file(wfs_url)

In [9]:
# link nvcl holes to the more detailed drillhole records and extract the report IDs
boreholes = []

for item in reader.borehole_list:
    reportids= None
    try:
        reportids=gdf[gdf['holeidentifier'] == item.nvcl_id]['reports'].values[0]
    except Exception as e:
        print(item.nvcl_id+f" has no reports")    

    feature = {"name": item.name , "latitude": item.y, "longitude": item.x, "reportids": reportids }
    boreholes.append(feature)



COAL_027746 has no reports
MIN_001622 has no reports
MIN_007415 has no reports
MIN_007619 has no reports
MIN_010856 has no reports
MIN_010866 has no reports
MIN_010886 has no reports
MIN_010889 has no reports
MIN_013116 has no reports
MIN_013117 has no reports
MIN_016743 has no reports
MIN_016747 has no reports
MIN_017200 has no reports
MIN_017201 has no reports
MIN_020465 has no reports
MIN_024275 has no reports
MIN_024519 has no reports
MIN_024547 has no reports
MIN_024548 has no reports
MIN_024549 has no reports
MIN_025242 has no reports
MIN_025244 has no reports
MIN_027275 has no reports
MIN_027315 has no reports
MIN_027316 has no reports
MIN_027910 has no reports
MIN_028741 has no reports
MIN_032909 has no reports
MIN_040006 has no reports
MIN_040009 has no reports
MIN_040522 has no reports
MIN_040669 has no reports
MIN_040670 has no reports
MIN_040807 has no reports
MIN_040809 has no reports
MIN_040810 has no reports
MIN_042422 has no reports
MIN_042686 has no reports
MIN_043865 

In [10]:

# Create a base map
m = folium.Map(location=[-33, 145], zoom_start=7)

# display pins blue for holes with reports and red for holes without.  Encode a link to NSW's reports website to allow clicking through to the reports.

for item in boreholes:
    folium.Marker(
        location=[item["latitude"], item["longitude"]],
        popup= '<p>BoreholeName: '+item["name"]+'</p>'+('<p><a href="https://search.geoscience.nsw.gov.au/advanced?q=mr_rin:('+item["reportids"]+')&t=digs">Reports</a></p>' if (item["reportids"]!=None) else '' ),
        icon=(folium.Icon(color="blue") if (item["reportids"]!=None) else folium.Icon(color="red"))
    ).add_to(m)

# Save the map to an HTML file
m.save("map.html")

m